In [21]:
import numpy as np
import astropy.io.fits as pyfits
import matplotlib.pyplot as plt
import json
import pandas

In [30]:
input_file = "NGC7469.fits"
targ_name = "NGC7469"
aperture_radius = 15 # arcsec
data_dir = "/Users/dkakkad/work/research_projects/BASS/muse_cubes/"

In [31]:
def wl_cube_muse(fits_cube):
    cube = pyfits.open(fits_cube)
    crpix = cube[1].header["CRPIX3"]
    crval = cube[1].header["CRVAL3"]
    cdel  = cube[1].header["CD3_3"]
    num = cube[1].header["NAXIS3"]
    wl = (np.linspace(1,num,num) - crpix)*cdel + crval
    return wl

In [32]:
def aper_circ(img_data, x0, y0, R):
    Ny,Nx = np.shape(img_data)
    x,y = np.arange(Nx),np.arange(Ny)
    xgrid, ygrid = np.meshgrid(x,y)
    rgrid = ((xgrid-x0)**2+(ygrid-y0)**2)**0.5
    w = np.where(rgrid < R)
    flux = np.sum(img_data[w])#.sum()
    return flux

In [33]:
targ_cube = pyfits.open(data_dir+input_file)
wl = wl_cube_muse(data_dir+input_file)

sci_data = targ_cube[1].data
Nz,Ny,Nx = np.shape(sci_data)

In [34]:
with open("phot_bands.json","r") as read_file:
    data = json.load(read_file)
    
phot_bands = data['phot_bands']

In [35]:
# Iterate through all the photometric bands

new_hdul = pyfits.HDUList()
new_hdul.append(pyfits.ImageHDU([0], name="Primary"))
for i in range(0,len(phot_bands)):
    #print("Working on "+phot_bands[i]['band'])
    cenwave = phot_bands[i]['cenwave']
    bandwidth = phot_bands[i]['bandwidth']
    filt_wl = np.where((wl>cenwave-bandwidth/2)&(wl<cenwave+bandwidth/2))
    
    filt_img = [[np.mean(sci_data[:,j,i][filt_wl]) for j in range(0,Ny)] for i in range(0,Nx)]
    filt_img = np.array(filt_img)
    new_hdul.append(pyfits.ImageHDU(filt_img, header=targ_cube[1].header, name=phot_bands[i]['band']))
    
    filt_img[np.isnan(filt_img)] = 0
    
    y0,x0 = np.where(filt_img == np.max(filt_img))
    
    #plt.imshow(filt_img, origin='lower')
    
    f_lambda = aper_circ(filt_img,x0[0],y0[0],int(aperture_radius/0.2))*1.0e-20 # erg/s/cm2/A
    
    # erg/s/cm2/Hz; formula from https://hea-www.harvard.edu/~pgreen/figs/Conversions.pdf
    f_nu_Jy = 3.34e+4*cenwave*cenwave*f_lambda # Jy
    
    data = {"Band":phot_bands[i]['band'],"Flux [mJy]":"{:.2f}".format(f_nu_Jy*1000)}
    
    print(phot_bands[i]['band']+" = "+"{:.2f}".format(f_nu_Jy*1000)+" mJy") # mJy
    
new_hdul.writeto("MUSE_photometry_"+targ_name+".fits",overwrite=True)

B = 17.88 mJy
G = 20.68 mJy
V = 26.80 mJy
R = 41.68 mJy
